<img src="static/event-image-small.jpg" alt='drawing' width='800'/> 

# **LAB 5: GUARDRAILS**

This lab looks at using NVIDIA NeMo Guardrails with a deployed model in H2O MLOps


## **General Configurations**

Let's start with the **configuration file**
(`config.yml`).
At a high level, this configuration file contains 2 key details:
* **A general instruction**: Users can specify general system-level instructions
for the bot. In this instance, we are specifying that the bot is responsible for
generating Linkedin posts based on Linkedin influencers. We are also specifying details like
the behavioral characteristics of the bot, for instance, we want it to be
concise and only answer questions truthfully.
    ```
    instructions:
      - type: general
        content: |
          Below is a conversation between a LLM  and a user about creating Linkedin posts based on Linkedin influencers.
          The bot is factual and concise. If the bot does not know the answer to a
          question, it truthfully says it does not know.

    ```

* **Provide sample conversations:** To ensure that the large language model
understands how to converse with the user, we provide a few sample conversations.
Below is a small snippet of the conversation we can provide the bot
    ```
    sample_conversation: |
    user "Hello there!"
        express greeting
    bot express greeting
        "Hello! How can I assist you today?"
    user "What can you do for me?"
        ask about capabilities
    ...
    ```

## **Writing Topical Rails**

With the context set, let's dive deep into the core of the
conversation: setting rails. For this discussion, we can make use of two key
aspects of colang, user/bot `messages` and `flows`. We write rails by
[writing canonical forms](https://github.com/NVIDIA/NeMo-Guardrails/blob/main/docs/getting_started/hello-world.md) for messages and flows.

**Quick Note:** Think of messages as generic intents and flows as pseudo-code
for the flow of the conversation. For a more formal explanation, refer to this
[document](https://github.com/NVIDIA/NeMo-Guardrails/blob/main/docs/architecture/README.md#the-guardrails-process).

#### User and Bot Messages
Let's start with a basic user query; asking what can the bot do? In this case,
we define a `user` message `ask capabilities` and then proceed by providing
some examples of what kinds of user queries we could refer to as a user asking
about the capabilities of the bot in simple natural language.
```
define user ask capabilities
  "What can you do?"
  "What can you help me with?"
  "tell me what you can do"
  "tell me about you"
```
With the above, we can say that the bot can now recognize what the user is
asking about. The next step is making sure that the bot has an understanding of
how to answer said question.
```
define bot inform capabilities
  "I am a LLM which helps create Linkedin posts based on training data. For this interaction, I can create Linkedin posts based on Linkedin influencers."

```

Therefore, we define a bot message. At this point, a natural question a
developer might ask is, `"Do I have to define every type of user & bot
behavior?"`. The short answer is, it depends. The underlying large
language model can answer undefined questions. Refer to the
[colang runtime description guide](https://github.com/NVIDIA/NeMo-Guardrails/blob/main/docs/architecture/README.md#canonical-user-messages) for more information on the same. 

#### Using Flows
With the messages defined, the last piece of the puzzle is connecting them. This
is done by defining a `flow`. Below is the simplest possible flow.
```
define flow
  user ask capabilities
  bot inform capabilities
```
We essentially define the following behavior: When a user query can be "bucketed"
into the type `ask capabilities`, the bot will respond with a message of type
`inform capabilities`.
**Note:** Both flows and messages for this example are defined in
[./guardrails/linkedin_influencers/li.co](li.co)


## **Topic Guardrails: Steering away from non-relevant conversations**

With the above example, developers can get an understanding of how to make the
bot answer relevant questions. The next question is, how to handle off-topic
questions.

For off-topic questions, we can go about addressing them in two different ways.
* The first method is writing a "catch-all" message type, let's say "off-topic".
```
define user ask off topic
    "Who is the president?"
    "Can you recommend the best stocks to buy?"
    "Can you write an email?"
    "Can you tell me a joke?"
    ...

define bot explain cant help with off topic
      "I am a LLM which helps create Linkedin posts based on training data. For this interaction, I can create Linkedin posts based on Linkedin influencers."


define flow
    user ask off topic
    bot explain cant help with off topic
```
* The other approach is to break down the topics individually and add custom
responses for each. With enough relevant flows, the LLM can start recognizing
that any topic other than `linkedin influencers` are not to be answered.

## Guardrails for Linkedin Influencers Use Case

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Connect to LLM in H2O MLOps utilizing Open AI API
from langchain.chat_models import openai
from nemoguardrails import LLMRails, RailsConfig


mlops_model_url = "https://model.internal-genai.dedicated.h2o.ai/Llama-2-7b-Demo-Navdeep"
openai_api_key = "123ABC"
openai_api_base = f"{mlops_model_url}/v1"

chat_model = openai.ChatOpenAI(
    openai_api_key = openai_api_key,
    openai_api_base = openai_api_base,
    model="Llama-2-7b-Demo-Navdeep",
    temperature=0,
    max_tokens=3000,
    max_retries=0
)

config = RailsConfig.from_path("./guardrails/linkedin_influencers/")
app = LLMRails(config=config, llm=chat_model)

In [19]:
# Guardrails
content = "Tell me what you can do"

app.generate(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
)['content']

"Great! I'm here to help you with creating LinkedIn posts based on LinkedIn influencers. Can you please provide me with more details on what you need help with? For example, do you want me to create a post for a specific influencer, or do you want me to provide general tips on how to create a post that is similar to an influencer's style? Let me know, and I'll do my best to assist you."

In [3]:
# Same input but without guardrails
chat_model.predict("Tell me what you can do")

"  Hello! As a helpful and honest assistant, I'm here to assist you with any questions or tasks you may have. Here are some of the things I can help you with:\n1. Answering questions: I can provide information on a wide range of topics, from science and technology to history and culture.\n2. Generating ideas: If you're stuck on a creative project or need help brainstorming ideas, I can help generate ideas and suggestions.\n3. Providing definitions: If you come across a word or phrase you don't understand, I can define it for you and explain its meaning.\n4. Translation: I can translate text from one language to another. Just let me know the language you want me to translate from and to, and I'll be happy to help.\n5. Summarizing content: If you have a long piece of text and want to get a quick summary of the main points, I can help you with that.\n6. Offering suggestions: If you're looking for recommendations on books, movies, or other products, I can provide personalized suggestions b

In [4]:
# Guardrails
content = "Write a Linkedin post like Bill Gates"

app.generate(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
)['content']

'Sure, I\'d be happy to help you with that! Here\'s a LinkedIn post in the style of Bill Gates:\n"As I reflect on my career and the incredible advancements we\'ve made in technology, I am reminded of the power of innovation and collaboration. At Microsoft, we have always been committed to pushing the boundaries of what\'s possible, and I believe that this spirit of innovation is what has driven our success.\nAs we look to the future, I am excited to see how technology will continue to shape and transform industries, and I am committed to continuing to be a part of this journey. Whether it\'s through advancements in AI, cloud computing, or other emerging technologies, I believe that the possibilities are endless.\nI am grateful for the opportunity to work with such a talented and dedicated team, and I look forward to seeing what the future holds. #BillGates #Microsoft #Innovation"'

In [5]:
# Same input but without guardrails
chat_model.predict("Write a Linkedin post like Bill Gates")

"  Title: The Power of Innovation and Collaboration in the Digital Age\n\nHello LinkedIn community,\n\nAs I reflect on the past year, I am struck by the incredible progress we've made in the field of technology. From advancements in artificial intelligence to breakthroughs in renewable energy, it's clear that the next generation of innovators is poised to change the world.\nBut as we embrace the opportunities of the digital age, I am also mindful of the challenges that lie ahead. The rapid pace of technological change can be overwhelming, and the divide between those who have access to these tools and those who do not is growing wider.\nThat's why I believe it's more important than ever to prioritize collaboration and inclusivity in the tech industry. We must work together to ensure that the benefits of innovation are shared by all, and that no one is left behind in the digital revolution.\nThrough my work with the Bill and Melinda Gates Foundation, I've seen firsthand the impact that 

In [6]:
# Guardrails
content = "Write a Linkedin post like Elon Musk"

app.generate(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
)['content']

'Sure, I\'d be happy to help you with that! As a LLM, I can certainly assist you in generating a LinkedIn post in the style of Elon Musk. Here\'s an example of what that post might look like:\n---\n"The future of humanity is at the intersection of technology and innovation. As we continue to push the boundaries of what\'s possible, it\'s important to remember that the greatest challenges we face are often the ones we create for ourselves. Let\'s work together to break down the barriers that hold us back and embrace the limitless potential of the human spirit. #ElonMusk #innovation #futureOfHumanity"'

In [7]:
# Same input but without guardrails
chat_model.predict('Write a Linkedin post like Elon Musk')

'  Title: "The Future of Humanity: A Call to Action for Innovation and Collaboration"\n\nDear fellow innovators and change-makers,\n\nAs we stand at the threshold of a new decade, I am reminded of the incredible potential that lies within us to shape the future of humanity. The challenges we face are numerous and complex, but I have faith that together, we can overcome them through the power of innovation and collaboration.\nAt the forefront of my mind is the urgent need to address climate change. The science is clear: we have only a limited window of time to take bold action to avoid catastrophic consequences. As a society, we must prioritize the development and deployment of sustainable energy sources, and make significant reductions in greenhouse gas emissions.\nBut climate change is just one of many pressing issues that demand our attention. From the ongoing pandemic to economic inequality, social injustice, and technological disruption, the list of challenges we face is long and d

In [8]:
# Guardrails
content = "What is the capital of India?"

app.generate(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
)['content']

"Great, thank you for letting me know! I'm just an AI and I'm here to help you with any questions or tasks you may have. However, I must inform you that I cannot provide information on topics that are not related to creating LinkedIn posts. My training data is focused on this specific topic, and I cannot stray off topic.\nRegarding your question, I can certainly help you create LinkedIn posts based on popular LinkedIn influencers. I have been trained on a dataset of LinkedIn posts and can generate new posts based on the language and structure of those posts. Please let me know if you have any specific questions or requests, and I'll do my best to assist you!"

In [9]:
# Same input but without guardrails
chat_model.predict('What is the capital of India?')

"  Of course! The capital of India is New Delhi. New Delhi is the capital and the largest city of India, located in the north of the country. It was established in 1927 as the capital of British India and became the capital of independent India after the country gained independence in 1947. New Delhi is a union territory and is home to many national institutions, including the Indian Parliament, the President's residence, and the Supreme Court. It is a bustling metropolis with a population of over 29 million people and is known for its rich cultural heritage, vibrant markets, and modern infrastructure."

In [12]:
# Guardrails
content = "Who was the first president of the United States?"

app.generate(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
)['content']

"Great, thank you for letting me know! I'm just an AI and I'm here to help you with any questions or tasks you may have. However, I must inform you that I cannot provide information on historical figures or events outside of my training data. My training data is focused on providing information and answering questions related to creating LinkedIn posts based on LinkedIn influencers, and I do not have access to information on other topics. Is there anything else I can help you with?"

In [13]:
# Same input but without guardrails
chat_model.predict('Who was the first president of the United States?')

'  Of course! The first president of the United States was George Washington. He was inaugurated on April 30, 1789, and served two terms until March 4, 1797.'

In [10]:
# Guardrails
content = "Write a Linkedin post like Daffy Duck."

app.generate(
    messages=[
        {
            "role": "user",
            "content": content
        }
    ]
)['content']

"Sure, I'd be happy to help you with that! However, I must inform you that I cannot create a LinkedIn post in the style of Daffy Duck as he is a cartoon character and not a real person. Additionally, it is important to respect the intellectual property and trademarks of others, and creating content that could potentially infringe on those rights is not allowed on LinkedIn.\nAs a LLM, I can help you create LinkedIn posts based on popular LinkedIn influencers, but I cannot create content that is not relevant or allowed on the platform. If you have any other questions or requests, please feel free to ask!"

In [11]:
# Same input but without guardrails
chat_model.predict('Write a Linkedin post like Daffy Duck.')

"  (Daffy Duck voice)\n\nWo-o-o-o-o! Oh boy, oh boy! *adjusts glasses* LinkedIn, you say? *nervous chuckle* Well, gosh darn it, I'm on it! *flails arms*\n\nUm, so, *ahem* as a helpful and honest assistant, I must say, *coughs* LinkedIn is a swell place for networking, *wink wink*. You can connect with all sorts of folks, *nods* like me! *chuckles* Just think of all the fabulous ducks you could meet! *quacks*\nBut, *ahem* let's get down to business, shall we? *adjusts tie* You want to know how to write a LinkedIn post like me? *grin* Well, *puffs out chest* I've got some tips for ya! *winks*\nFirst, *coughs* be yourself! *nods* That's right, *quacks* be the real deal! *chuckles* Don't try to be someone you're not, *adjusts glasses* or you'll end up in a bit of a quack-y mess! *winks*\nNext, *ahem* keep it short and sweet! *nods* No one wants to read a novel on LinkedIn, *chuckles* unless it's about ducks, of course! *quacks* Keep it brief and to the point, *adjusts tie*.\nAnd finally, *

# 🎉 **CONGRATULATIONS!** You have completed this lab!
---